In [27]:
from flask import current_app, request
from mastodon import Mastodon
import json, time
import pandas as pd
from textblob import TextBlob
import re
from bs4 import BeautifulSoup

In [48]:
def extract_text(html_content):
    html_content = re.sub(r'href=\S+', '', html_content)
    # html_content = re.sub(r'http\S+', '', html_content)
    # html_content = re.sub(r'["\']', '', html_content)
    # html_content = re.sub(r'[^\w\s]', '', html_content)    

    # Create a BeautifulSoup object and specify the parser
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Remove elements that are meant to be invisible or are not relevant to text content
    for element in soup.find_all("span", class_="invisible"):
        element.decompose()  # This removes the element from the soup

    # Extract text from the modified HTML
    text = soup.get_text(separator=' ', strip=True)
    return text

In [16]:
def main():
    m = Mastodon(
        api_base_url=f'https://mastodon.au'
    )

    # Get the ID of the lastid status main the public timeline
    lastid= m.timeline(timeline='public', since_id=None, limit=1, remote=True)[0]['id']

    # Sleep for 5 seconds to allow for some status to be posted before we fetch them
    time.sleep(5)

    # Fetch the statuses from the public timeline since the lastid
    # return json.dumps(m.timeline(timeline='public', since_id=lastid, remote=True), default=str)

    m_data = m.timeline(timeline='public', since_id=lastid, remote=True)

    res = []
    for data in m_data:
        res.append({
            'created_at': data['created_at'],
            'content': data['content'],
            # 'sar_age': data['sar_age'],
            }
        )

    return res


In [55]:
result = main()
df = pd.DataFrame(result)


In [56]:
df

,created_at,content
0,2024-05-10 08:57:56+00:00,<p>Having to squint at microscopic ingredient ...
1,2024-05-10 08:57:58+00:00,"<p>New tariffs on Chinese EVs, other sectors e..."


In [57]:
df['content'] = df['content'].apply(extract_text)
df

,created_at,content
0,2024-05-10 08:57:56+00:00,Having to squint at microscopic ingredient lis...
1,2024-05-10 08:57:58+00:00,"New tariffs on Chinese EVs, other sectors expe..."


In [59]:
df['sentiment'] = df['content'].apply(lambda x: TextBlob(x).sentiment.polarity)
df

,created_at,content,sentiment
0,2024-05-10 08:57:56+00:00,Having to squint at microscopic ingredient lis...,0.321429
1,2024-05-10 08:57:58+00:00,"New tariffs on Chinese EVs, other sectors expe...",-0.017727


In [33]:
blob = TextBlob(extract_text(df['content'][1]))
blob.sentiment.polarity

0.0

In [49]:
extract_text(df['content'][1])

'ca.movies.yahoo.com/news/resea # archaeology # scotland (Researchers uncover Neolithic culinary traditions.)'

In [50]:
extract_text("href=\"https://cloudisland.nz/tags/StarWa\" So, I'm on this Shinkansen train in Japan ...")

"So, I'm on this Shinkansen train in Japan ..."